[View in Colaboratory](https://colab.research.google.com/github/mforbes/wsu_physics_450/blob/master/Notebooks/Particle_in_a_Box.ipynb)

# Particle in a Box

$$
  \newcommand{\ket}[1]{\lvert#1\rangle}
  \newcommand{\bra}[1]{\langle#1\rvert}
  \newcommand{\braket}[1]{\langle#1\rangle}  
  \newcommand{\Ket}[1]{\left\lvert#1\right\rangle}
  \newcommand{\Bra}[1]{\left\langle#1\right\rvert}
  \newcommand{\Braket}[1]{\left\langle#1\right\rangle}
  \newcommand{\op}[1]{\hat{\boldsymbol{#1}}}
  \newcommand{\mat}[1]{\boldsymbol{#1}}
  \newcommand{\I}{\mathrm{i}}
  \newcommand{\d}{\mathrm{d}}
  \newcommand{\Floor}[1]{\left\lfloor\!#1\!\right\rfloor}
$$


Here we solve the problem of a particle in a box as discussed in class.  The Schrödinger equation has the form

$$
  \I \hbar \dot{\psi}(x, t) = \frac{-\hbar^2\psi''(x, t)}{2m} + V(x, t)\psi(x, t).
$$

A particle in a box has the following potential:

$$
  V(x, t) = \begin{cases}
    0 & 0<x<L,\\
    \infty & \text{otherwise}.
  \end{cases}
$$

The eigenstates are:

$$
  \psi_n(x) = \braket{x|n} = \sqrt{\frac{2}{L}}\sin(k_n x), \qquad
  k_n = \left.\frac{\pi n}{L}\right|_{n=1}^{\infty}, \qquad
  E_n = \frac{\hbar^2k_n^2}{2m}.
$$

These satisfy the boundary conditions that $\psi(0)= \psi(L) = 0$ and the SEQ in the interior.

## Sudden Expansion

In class, we considered the problem of what happens to the ground state $\ket{1}$ when the box is suddenly changed to have length $2L$.  To deal with this, we first express the eigenstates $\ket{n'}$ of box of length $2L$:

$$
  \braket{x|n'} = \sqrt{\frac{2}{2L}}\sin(k_{n'} x) 
                         = \sqrt{\frac{1}{L}}\sin(k_{n'} x), \qquad
  k_{n'} = \left.\frac{\pi n'}{2L}\right|_{n'=1}^{\infty}, \qquad
  E_{n'} = \frac{\hbar^2k_{n'}^2}{2m} = \frac{\hbar^2 \pi^2}{8mL^2}{n'}^2.
$$

The strategy is to express the initial state $\ket{1}$ in terms of the new eigenstates $\ket{n'}$ and then to follow the time evolution in the new eigenbasis of the Hamiltonian in the expanded box:

$$
  \ket{1, t=0} = \sum_{n'}\ket{n'}\braket{n'|1}
                       = \sum_{n'}\ket{n'}\int_{0}^{L} \sqrt{\frac{2}{L}}\sqrt{\frac{1}{L}}\sin(k_{n'}x)\sin(k_1x)\d{x}
                       = \frac{\sqrt{2}}{L}\sum_{n'}\ket{n'}\int_{0}^{L} \sin\frac{\pi n' x}{2L}\sin\frac{\pi x}{L}\d{x} = \\
  = \frac{\sqrt{2}}{\pi}\sum_{n'}\ket{n'} \int_{0}^{\pi} \sin\frac{n'\theta}{2}\sin\theta\d{\theta}.
$$

The integral can be evaluated by converting to exponential form $\sin\theta = \frac{e^{\I\theta}-e^{-\I\theta}}{2}$, and after some algebra, we have:

$$
  \ket{1, t=0} = \frac{\sqrt{2}}{\pi}\sum_{n'}\ket{n'}\frac{4\sin\frac{n'\pi}{2}}{4-{n'}^2}
                       = \frac{\ket{2'}}{\sqrt{2}} + \frac{\sqrt{2}}{\pi}\sum_{n' \text{ odd}}^{\infty}\ket{n'}\frac{4(-1)^{(n'-1)/2}}{4-{n'}^2}\\
  = \frac{\ket{2'}}{\sqrt{2}} 
      + \frac{4\sqrt{2}}{\pi}\left(
      - \frac{\ket{1'}}{3\times (-1)}
     + \frac{\ket{3'}}{5\times 1} 
     - \frac{\ket{5'}}{7\times 3} 
     + \frac{\ket{7'}}{9\times 5} 
     - \frac{\ket{9'}}{11\times 7}
     %+ \frac{\ket{11'}}{13\times 9}
     %- \frac{\ket{13'}}{15\times 11}
     + \dots
     \right)
$$

All even terms vanish except for $n'=2$ which has the specified limiting value.

In [0]:
%pylab inline --no-import-all
N_terms = 5000
L = 1.0
x = np.linspace(0, 2*L, 100)   # Abscissa for plotting
n = np.arange(N_terms)         # Odd n'
c_n = np.sqrt(2)/np.pi * np.where(n%2 == 0, 0, 4*(-1)**((n-1)/2)/(4-n**2))
c_n[2] = 1./np.sqrt(2)

assert np.allclose(sum(c_n**2), 1)  # Check normalization
x_ = x[None, :]
n_ = n[:, None]
c_n_ = c_n[:, None]
hbar = m = 1.0
k_n_ = (np.pi*n_/2/L)
E_n_ = (hbar*k_n_)**2/2/m

def get_psi(t):
  return ((np.exp(E_n_*t/1j/hbar)*c_n_) * np.sin(k_n_*x_)).sum(axis=0)

plt.plot(x, abs(get_psi(t=0))**2)

Here is an animation based on [this discussion](https://medium.com/lambda-school-machine-learning/making-animations-work-in-google-colaboratory-new-home-for-ml-prototyping-c6147186ae75).  Use the sliders to adjust the number of times to display and maximum time.  Note: it can take quite a while for the results to generate if these are large.

In [0]:
#@title Particle in a Box { run: "auto", display-mode: "form" }
from matplotlib import animation, rc
from IPython.display import HTML, clear_output
from matplotlib.patches import Rectangle

tau = 16*m*L**2/hbar/np.pi

# animate over some set of ts
Nt = 128         #@param {type:"slider", min:0, max:1000, step:2}
t_max_tau = 0.5  #@param {type:"slider", min:0.1, max:1.0, step:0.1}
t_max = t_max_tau * tau
ts = np.linspace(0, t_max, Nt+1)

# First set up the figure, the axes, and the plot element
fig, ax = plt.subplots()
plt.close()
ax.set_xlim((-0.1, 2*L+0.1))
ax.set_ylim((0, 2.3))
_rect_args = dict(width=0.1, height=2.3, hatch='\\', fill=False)
ax.add_patch(Rectangle((-0.1,0), **_rect_args))
p = ax.add_patch(Rectangle((L,0.), **_rect_args))

line, = ax.plot([], [], lw=2)
title = ax.set_title(r"$t/\tau=0$")

# initialization function: plot the background of each frame
def init():
    line.set_data(x, abs(get_psi(t=0))**2)
    return (line,)
  
# animation function: this is called sequentially
def animate(i):
    t = ts[i] 
    # Update data
    line.set_data(x, abs(get_psi(t=t))**2)
    if i > 0:
        # Move right wall after first step.
        p.set_xy((2*L, 0))
        title.set_text(r"$t/\tau={:.4f}$".format(t/tau))
    return (line,)

anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=len(ts), 
    interval=50, repeat=False, blit=True)
plt.close('all')
rc('animation', html='jshtml')
anim


## Comments and Questions

Playing with the previous simulation, you might notice a couple of interesting aspects.

* All of the dynamics comes from simply changing the relative phases $\delta_{n'}(t)$ of the various terms in the sum:

  $$
    \ket{1, t} = \sum_{n'}e^{-\I\delta_{n'}(t)}\ket{n'}c_{n'}, \qquad
    \delta_{n'}(t) = \frac{E_{n'}}{\hbar}, \qquad
    c_{n'} = \frac{4\sin\frac{n'\pi}{2}}{4-{n'}^2}\\
    \psi(x, t) = \sum_{n'}e^{-\I\delta_{n'}(t)}c_{n'}\sin\frac{\pi n' x}{2L}.
  $$
  
  There is no time-dependence in the actual basis states - all dynamics comes from interference between the various terms.
  
* How does the energy of the system change as the wall is moved?
  
* High-frequency ripples appear rapidly due to the sudden motion of a sharp boundary.  If we were to move the boundary more smoothly, there would be fewer such excitations.

* There seem to be special times when the ripples in the system dissapear leaving a smooth curve.  This is because the eigen-energies can be expressed as:

  $$
    \frac{E_{n'}}{\hbar}t =
    2\pi {n'}^2 \frac{t}{\tau}, \qquad
    \tau = \frac{16mL^2}{\hbar \pi}.
  $$
  
  Thus, when $t=\tau$, all the phases are unity and we return to the original state.  At $t=\tau/2$ the phases are real, but the signs of the terms change so that the wavefunction has moved symmetrically to the right side of the box, and at time $t\in\{\tau/4, 3\tau/4\}$ the wavefunction symmetrically occupies the box with a node at $x=L$.  The states at $t\in\{\tau/8, 3\tau/8, 5\tau/8, 7\tau/8\}$ also have nodes at $x=L$ and a smooth but asymmetric form.
  
  At times $n\tau/16$, the density has sharp features.  These are related to similar artifacts such as plateaus (perfectly flat regions) seen when the box is expanded to $1.5L$ (see [C. Aslangul (2008): "Surprises in the suddenly-expanded infinite well"](http://dx.doi.org/10.1088/1751-8113/41/7/075301) [[arXiv:0709.1101](https://arxiv.org/abs/0709.1101)]).

In [0]:
for n in range(0, 8):
  plt.plot(x, abs(get_psi(n*tau/8))**2)

In [0]:
for n in [3, 7, 11, 15]:
  plt.plot(x, abs(get_psi(n*tau/16))**2)